# Machine Learning Evaluation of NGS Fastqc Results

## Import fastqc results into a dataframe

# Import fastqQCParser   
https://pypi.org/project/fastqcparser/#description

In [1]:
from pprint import pprint
from fastqcparser import FastQCParser

In [2]:
import pandas as pd
import os

In [3]:
rootdir = 'training_data'

In [4]:
def import_reads(rootdir):
    reads = []
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            filepath = root + os.sep + name
            if filepath.endswith("fastqc_data.txt"):
                reads.append(FastQCParser(filepath))
    return reads

In [5]:
reads = import_reads(rootdir)

In [6]:
len(reads)

184

In [7]:
module_list = list(reads[0].modules.keys())
# imported reads don't include the module 'Overrepresented sequences' 
# if there are none in the read, so we manually add the module to the list
if(len(module_list) < 11):
    module_list.insert(9, 'Overrepresented sequences')
module_list.append('Module Statuses')
module_list.append('filename')
print(module_list)

['Basic Statistics', 'Per base sequence quality', 'Per tile sequence quality', 'Per sequence quality scores', 'Per base sequence content', 'Per sequence GC content', 'Per base N content', 'Sequence Length Distribution', 'Sequence Duplication Levels', 'Overrepresented sequences', 'Adapter Content', 'Module Statuses', 'filename']


In [8]:
def create_single_read_dataframe(result):
    module_result = []
    module_status = []
    
    for module in result.modules:
        result_data = pd.DataFrame(result.modules[module]['data'])
        result_data.columns = result.modules[module]['fieldnames']
        module_status.append(result.modules[module]['status'])
        module_result.append(result_data)
    # imported reads don't include the module 'Overrepresented sequences' 
    # if there are none in the read, so we manually add the status list
    # and an empty dataframe /is None better?
    if len(module_status) < 11:
        module_status.insert(9, 'pass')
        module_result.insert(9, pd.DataFrame())
    module_result.append(module_status)
    module_result.append(module_result[0].Value[0].replace(".fastq.gz", ""))
    #print(len(module_result), ' - ',  module_result[0].Value[0], '-', len(module_list) )
    module_series = pd.Series(data=module_result, index=module_list)
    single_read = module_series.to_frame().T#.set_index('filename')
    return single_read

In [9]:
read_list = []
for read in reads:
    single_read = create_single_read_dataframe(read)
    read_list.append(single_read)

In [10]:
read_results = pd.concat(read_list)
#read_results

In [11]:
pd.set_option('display.max_rows', 3)

In [12]:
metadata = pd.read_json('exported_datasets/metadata.json')

In [13]:
complete_reads = pd.merge(read_results, metadata, on='filename', how='inner').set_index('filename')
complete_reads

,Basic Statistics,Per base sequence quality,Per tile sequence quality,Per sequence quality scores,Per base sequence content,Per sequence GC content,Per base N content,Sequence Length Distribution,Sequence Duplication Levels,Overrepresented sequences,Adapter Content,Module Statuses,organism,technology,read_number,evaluation
filename,,,,,,,,,,,,,,,,
200709_20-07968_20-00891_S21_L000_R2_001,Measure ...,Base Mean Median Lower Quartile...,Tile Base Mean 0 1101 ...,Quality Count 0 2 3.0 .....,Base G A T ...,GC Content Count 0 0 3.0 ...,Base N-Count 0 1 0.479607 .....,Length Count 0 30-39 4192.0 .. ...,Duplication Level Percentage of deduplicat...,Empty DataFrame Columns: [] Index: [],Position Illumina Universal Adapter Illum...,"[pass, fail, pass, pass, fail, pass, pass, war...",Sau,MS,2,ugly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191113_19-10151_20475_S133_L000_R1_001,Measure ...,Base Mean Median Lower Quartile...,Tile Base Mean 0 1101 ...,Quality Count 0 16 2.0 .....,Base G A T ...,GC Content Count 0 0 0.0 ...,Base N-Count 0 1 0.083286 .....,Length Count 0 35-39 4284.0 .....,Duplication Level Percentage of deduplicat...,Empty DataFrame Columns: [] Index: [],Position Illumina Universal Adapter Illum...,"[pass, pass, pass, pass, fail, pass, pass, war...",Efcm,HS,1,good


In [14]:
print('exported complete dataset')
complete_reads.to_json('exported_datasets/complete_set.json')

exported complete dataset
